In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde # Разрешено для сравнения, не для реализации алгоритма

# 1. Реализация алгоритма KDE с использованием только numpy/pandas

class SimpleKDE:
    """
    Простая реализация Kernel Density Estimation (KDE)
    с использованием только numpy и pandas.
    Использует Гауссовское ядро и правило Сильвермана для выбора полосы пропускания.
    """
    def __init__(self, bandwidth=None):
        """
        Инициализация KDE.

        Args:
            bandwidth (float, optional): Полоса пропускания (h). Если None,
                                         используется правило Сильвермана.
                                         По умолчанию None.
        """
        self.bandwidth = bandwidth
        self.data = None
        self.n = 0
        self.sigma = 0.0

    def fit(self, data):
        """
        Обучает модель KDE на данных.

        Args:
            data (np.ndarray или pd.Series): Входные данные (1D массив).
        """
        if not isinstance(data, (np.ndarray, pd.Series)):
            raise TypeError("Входные данные должны быть np.ndarray или pd.Series")
        if data.ndim != 1:
             raise ValueError("Входные данные должны быть 1D массивом")

        self.data = np.asarray(data)
        self.n = len(self.data)
        if self.n == 0:
            self.sigma = 0.0
            if self.bandwidth is None:
                self.bandwidth = 1.0 # Дефолтное значение для пустых данных
            return

        self.sigma = np.std(self.data)

        # Правило Сильвермана для выбора полосы пропускания (h)
        if self.bandwidth is None:
            # Избегаем деления на ноль или очень маленьких чисел, если std = 0
            if self.sigma < 1e-9:
                 # Если все точки одинаковы, выбираем небольшую полосу пропускания
                 self.bandwidth = 1.0
            else:
                self.bandwidth = (4 * self.sigma**5 / (3 * self.n))**(1/5)

        # Дополнительная проверка на очень маленькую полосу пропускания
        if self.bandwidth < 1e-9:
             self.bandwidth = 1e-9


    def _gaussian_kernel(self, u):
        """
        Гауссовское ядро.

        Args:
            u (np.ndarray): Входные значения для ядра.

        Returns:
            np.ndarray: Выходные значения ядра.
        """
        return (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * u**2)

    def predict(self, points):
        """
        Оценивает плотность в заданных точках.

        Args:
            points (np.ndarray или pd.Series): Точки, в которых нужно оценить плотность.

        Returns:
            np.ndarray: Оцененная плотность в заданных точках.
        """
        if self.data is None or self.n == 0:
            # Если данных нет, плотность везде равна 0
            return np.zeros_like(points, dtype=float)

        points = np.asarray(points)
        if points.ndim != 1:
             raise ValueError("Точки для оценки должны быть 1D массивом")

        # Применение формулы KDE: f(x) = (1/nh) * sum(K((x - xi) / h))
        density_estimates = np.zeros_like(points, dtype=float)

        for i, p in enumerate(points):
            # Вычисляем (x - xi) / h для всех точек данных
            u = (p - self.data) / self.bandwidth
            # Применяем ядро к u
            kernel_values = self._gaussian_kernel(u)
            # Суммируем значения ядра и масштабируем
            density_estimates[i] = np.sum(kernel_values) / (self.n * self.bandwidth)

        return density_estimates

# 2. Симуляция данных для тестирования

# Генерируем данные из нормального распределения
np.random.seed(42) # Для воспроизводимости
simulated_data = np.random.normal(loc=5, scale=2, size=100) # 100 точек, среднее 5, стд откл 2

# Генерируем точки, в которых будем оценивать плотность
evaluation_points = np.linspace(0, 10, 200) # 200 точек от 0 до 10

# 3. Сравнение со стандартной реализацией (scipy)

# Используем нашу реализацию
my_kde = SimpleKDE()
my_kde.fit(simulated_data)
my_density = my_kde.predict(evaluation_points)

# Используем стандартную реализацию из scipy
scipy_kde = gaussian_kde(simulated_data)
scipy_density = scipy_kde(evaluation_points)

# Выводим полосу пропускания, использованную нашей реализацией
print(f"Полоса пропускания (h) нашей реализации: {my_kde.bandwidth:.4f}")
print(f"Полоса пропускания (h) scipy реализации (оценка): {scipy_kde.factor * np.std(simulated_data):.4f}") # scipy.gaussian_kde.factor - это множитель для std

# Сравнение результатов
# Вычисляем среднеквадратичную ошибку (MSE) между двумя оценками плотности
mse = np.mean((my_density - scipy_density)**2)
print(f"Среднеквадратичная ошибка между нашей и scipy реализациями: {mse:.6f}")

# Визуализация (опционально, но полезно для проверки)
# Для визуализации потребуется matplotlib, который не входит в numpy/pandas.
# Если разрешено использовать matplotlib для визуализации, раскомментируйте следующий блок:

# import matplotlib.pyplot as plt
#
# plt.figure(figsize=(10, 6))
# plt.plot(evaluation_points, my_density, label='Наша реализация KDE')
# plt.plot(evaluation_points, scipy_density, label='scipy.stats.gaussian_kde')
# plt.hist(simulated_data, bins=30, density=True, alpha=0.5, label='Гистограмма данных')
# plt.title('Сравнение KDE: Наша реализация vs SciPy')
# plt.xlabel('Значение')
# plt.ylabel('Плотность')
# plt.legend()
# plt.grid(True)
# plt.show()

